In [1]:
from Crypto.Util.number import *
from tqdm import *
import itertools

def nextPrime(p):
    while(not isPrime(p)):
        p += 1
    return p

p = 680201537161531317827869565786140240595567913096417274637134403255116055511280864892266374758399999999999999999999999999999999999999999999999999999999999999999999999
deg1 = 2
deg2 = 5
leak1 = 84624382514957324426794167416980084161297449460045164807842311763375830274875400809588635343195174135691613055453493035516696630357254763624394674275492513550696448
leak2 = 569334021319485756763137861791243638993859221789562816154577308542976649646202822945301844698450579193326962681107954744030417590603938838204734858601313640848501344
leak3 = 325720000771917646719671745106544502680895911477018701616420509369836768451047103170212051953041518446572892754419417720965474892312189833039602545011787135282170400
leak4 = 607188653779811312711900086497209011406043384341389739547214249680956969386970129072753246609362372591781044704784040165947962309186896993435085198595013761765998592
n = 336631348442872227475735277623305104458216242371136122513108670525622121689052346348620824105820633714562546800934735845952338946486424402720093748392510487705996344393513464710255615634248631016778273746776059762111364616763243266453211367983670687473800622632626192654424964609849049774336197739675356042226277067017714668077407
cipher = 98106149415612242984147419198102021164901863054603625014502538888604192644391041028562541950192883864954991703087157292974235031070431515337932833347213492206351147110891983722997746801220179144171968380355798405953558533823282395282563952623698461579297014597381904081095895011393305504119244812232164909852287688815879984263470


##################################################### part1
F.<i> = GF(p^2, modulus = x^2 + 1)
PR.<a,d> = PolynomialRing(Zmod(p))

Find = 0
for b in trange(2^5):
    for c in range(2^5):
        a,d = PR.gens()
        X = (leak1 + a) + (leak2 + b) * i
        Y = (leak3 + c) + (leak4 + d) * i
        f = (
            X^3
            + Y^3
            - X^2 * Y^2
            + 1488 * X^2 * Y
            + 1488 * X * Y^2
            - 162000 * X^2
            - 162000 * Y^2
            + 40773375 * X * Y
            + 8748000000 * X
            + 8748000000 * Y
            - 157464000000000
        )

        f_re = 0
        f_im = 0
        for jj in f:
            temp = str(jj[0]).split("*i")
            if(len(temp) == 1):
                if(temp[0] == "i"):
                    f_im += jj[1]
                else:
                    f_re += int(temp[0])*jj[1]
            else:
                f_im += int(temp[0])*jj[1]
                if(temp[1] != ""):
                    f_re += int(temp[1][2:])*jj[1]


        h = f_im.sylvester_matrix(f_re, a).det().univariate_polynomial().monic()
        res = h.change_ring(Zmod(p)).roots()
        if(res != []):
            for jj in res:
                if(int(jj[0]) < 2^400):
                    Find = 1
                    j2il = int(jj[0])
                    j2 = (leak3 + c) + (leak4 + j2il) * i
                    break

                elif(p - int(jj[0]) < 2^400):
                    Find = 1
                    j2il = p - int(jj[0])
                    j2 = (leak3 + c) + (leak4 + j2il) * i
                    break
        if(Find):
            break
    if(Find):
        break


##################################################### part2
#Φ5
def find_neighbors_phi5(X,j_prev=None):
    R.<Y> = PolynomialRing(X.parent())
    Φ5 = (
        X^6
        + Y^6
        - X^5*Y^5 
        + 3720*X^5*Y^4 
        + 3720*X^4*Y^5 
        - 4550940*X^5*Y^3 
        - 4550940*X^3*Y^5
        + 2028551200*X^5*Y^2 
        + 2028551200*X^2*Y^5
        - 246683410950*X^5*Y
        - 246683410950*X*Y^5 
        + 1963211489280*X^5 
        + 1963211489280*Y^5 
        + 1665999364600*X^4*Y^4 
        + 107878928185336800*X^4*Y^3 
        + 107878928185336800*X^3*Y^4 
        + 383083609779811215375*X^4*Y^2 
        + 383083609779811215375*X^2*Y^4
        + 128541798906828816384000*X^4*Y 
        + 128541798906828816384000*X*Y^4  
        + 1284733132841424456253440*X^4
        + 1284733132841424456253440*Y^4
        - 441206965512914835246100*X^3*Y^3  
        + 26898488858380731577417728000*X^3*Y^2 
        + 26898488858380731577417728000*X^2*Y^3 
        - 192457934618928299655108231168000*X^3*Y
        - 192457934618928299655108231168000*X*Y^3 
        + 280244777828439527804321565297868800*X^3
        + 280244777828439527804321565297868800*Y^3
        + 5110941777552418083110765199360000*X^2*Y^2 
        + 36554736583949629295706472332656640000*X^2*Y 
        + 36554736583949629295706472332656640000*X*Y^2        
        + 6692500042627997708487149415015068467200*X^2 
        - 264073457076620596259715790247978782949376*X*Y 
        + 6692500042627997708487149415015068467200*Y^2 
        + 53274330803424425450420160273356509151232000*X 
        + 53274330803424425450420160273356509151232000*Y 
        + 141359947154721358697753474691071362751004672000
    )

    res = Φ5.roots(multiplicities=False)
    if(j_prev == None):
        return res
    else:
        return list(set(res) - set([j_prev]))
    

for j3 in find_neighbors_phi5(j2):
    p = GCD(nextPrime(int(j3[0])),n)
    if(p != 1):
        q = n // p
        d = inverse(65537,(p-1)*(q-1))
        print(long_to_bytes(int(pow(cipher,d,n))))
        break


#H&NCTF{S33m5_L1ke_C0pp3RsM1th_But_N0t:)}

 50%|█████████████████████████████████████████▌                                         | 16/32 [02:41<02:41, 10.09s/it]


b'H&NCTF{S33m5_L1ke_C0pp3RsM1th_But_N0t:)}'
